[The author's github-->](https://github.com/baranovxyz)

In [1]:
#!curl -LO http://files.grouplens.org/datasets/movielens/ml-100k.zip
#!unzip ml-100k.zip

http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

In [1]:
target_userid = 94

In [2]:
import os
import sys
from pyspark import SparkContext
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
import runpy
spark = runpy.run_path(os.path.join(spark_home, 'python/pyspark/shell.py'))
sc = spark['sc']

/usr/hdp/current/spark2-client/python/pyspark/shell.py:45: UserWarning: Failed to initialize Spark session.
  warnings.warn("Failed to initialize Spark session.")
Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/shell.py", line 41, in <module>
    spark = SparkSession._create_shell_session()
AttributeError: type object 'SparkSession' has no attribute '_create_shell_session'


SystemExit: 1

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from operator import add
import operator as op
import json
import re
from datetime import datetime
from pyspark.sql import Row
from pyspark.sql import functions as F
import numpy as np
import math
from IPython.display import clear_output

In [ ]:
!head -n3 ml-100k/u.data

The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC

Найдите количество всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).

In [ ]:
rdd_ratings = sc.textFile("/user/valery.baranov/u.data")\
    .map(lambda line: [int(d) for d in line.split('\t')])\
    .map(lambda ds: Row(userid=ds[0], movieid=ds[1], rating=ds[2], ts=ds[3]))

In [ ]:
df_ratings = rdd_ratings.toDF()

In [ ]:
df_ratings.show(n=3)

In [ ]:
num_users = rdd_ratings.map(lambda row: row.userid).distinct().count()
num_users

In [ ]:
def get_userid_iter():
    return range(1, num_users + 1)

In [ ]:
num_movies = rdd_ratings.map(lambda row: row.movieid).distinct().count()
num_movies

Сколько пользователь в среднем ставит рейтингов? Подсчитать `количество рейтингов / количество пользователей`. 
   Поле `average_user_ratings`.

In [12]:
num_ratings = rdd_ratings.count()
num_ratings

100000

In [13]:
average_user_ratings = num_ratings / num_users
average_user_ratings

106.04453870625663

Сколько фильм в среднем имеет рейтингов?  Подсчитать`количество рейтингов / количество фильмов`. 
   Поле `average_film_ratings`.

In [14]:
average_film_ratings = num_ratings / num_movies
average_film_ratings

59.45303210463734

Найдите процент заполненных ячеек в данных: `количество рейтингов / (количество пользователей * количество фильмов)`. 
   Поле `completeness`.

In [15]:
completeness = num_ratings / (num_users * num_movies)
completeness

0.06304669364224531

#1 Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/количество рейтингов пользователя). Здесь I<sub>a</sub> — множество фильмов, по которым у пользователя есть рейтинги r<sub>ui</sub>. Здесь и далее |I<sub>a</sub>| обозначает количество элементов в множестве I<sub>a</sub>.

<img width="100px" src="http://data.newprolab.com/public-newprolab-com/laba08_r_a_avg.png">

In [16]:
df_avg_user_rating = df_ratings.groupby('userid').mean('rating')

In [17]:
user_avg = df_avg_user_rating.rdd.map(lambda row: (row.userid, row['avg(rating)'])).collect()

In [18]:
user_avg[:3]

[(26, 2.94392523364486), (29, 3.6470588235294117), (474, 4.08256880733945)]

In [19]:
avg_ratings = np.array([v for k,v in sorted(user_avg)])
avg_ratings[:10]

array([ 3.61029412,  3.70967742,  2.7962963 ,  4.33333333,  2.87428571,
        3.63507109,  3.96526055,  3.79661017,  4.27272727,  4.20652174])

#2 Посчитайте меру близости Пирсона выданного вам пользователя со всеми остальными пользователями. Обратите внимание, что корреляция Пирсона считается только на пересечении, то есть вклад вносят только фильмы, оцененные совместно (I<sub>a</sub>, I<sub>u</sub> — множества оцененных пользователями `a` и `u` фильмов). Корреляция с константой (ситуация, когда у пользователя все оценки одинаковые) равна нулю.

Формат: `ID пользователя; корреляция Пирсона`.

<img width="350px" src="http://data.newprolab.com/public-newprolab-com/laba08_pearson.png">

In [20]:
df_pivot_ratings = df_ratings.groupBy("movieid").pivot("userid", range(1, num_users + 1)).sum("rating")\
    .coalesce(1).cache()

In [21]:
df_pivot_rows = df_ratings.groupBy("userid").pivot("movieid", range(1, num_movies + 1)).sum("rating").cache()

In [22]:
target_row = df_pivot_rows.rdd.filter(lambda row: row.userid == target_userid).collect()[0]
target_values =[v for k,v in sorted(target_row.asDict().items()) if k != 'userid']

In [23]:
len(target_values)

1682

In [24]:
target_values[:10]

[3, None, 4, None, None, None, None, None, None, None]

In [25]:
br_target_values = sc.broadcast(target_values)
br_avg_ratings = sc.broadcast(avg_ratings)

In [26]:
len(br_avg_ratings.value)

943

In [27]:
def pearson_byrows(row):
    row_ = row.asDict()
    userid = int(row_.pop('userid'))
    uid = userid - 1
    tid = target_userid - 1
    user_values =[v for k,v in sorted(row_.items())]
    ratings = [(user_values[i], br_target_values.value[i]) for i in range(len(br_target_values.value))\
               if user_values[i] is not None and br_target_values.value[i] is not None]
    
    data = [((v1 - br_avg_ratings.value[uid]),(v2 - br_avg_ratings.value[tid])) for v1, v2 in ratings]
        
    c = sum([v1*v2 for v1,v2 in data])
    z1 = sum([v1 ** 2 for v1,v2 in data])
    z2 = sum([v2 ** 2 for v1,v2 in data])
    if c == 0:
        return 0
    return (userid, c / (math.sqrt(z1) * math.sqrt(z2)))

In [28]:
row1 = df_pivot_rows.rdd.take(1)[0]

In [29]:
pearson_byrows(row1)

(474, 0.17761222077834857)

In [30]:
df_pivot_rows.rdd.map(pearson_byrows).take(3)

[(474, 0.17761222077834857),
 (26, 0.31266945349376479),
 (29, 0.21850782526734225)]

In [31]:
pearson_t = df_pivot_rows.rdd.map(pearson_byrows).collect()

In [32]:
pearson = [v for k,v in sorted(pearson_t)]

In [33]:
pearson[0]

0.40686121675947595

In [34]:
df_pivot_ratings.corr(str(1), str(target_userid), method='pearson')

0.3570300868991909

In [35]:
len(pearson)

943

#3. Посчитайте поправочный коэффициент для корреляции Пирсона на нехватку данных:

<img width="150px" src="http://data.newprolab.com/public-newprolab-com/laba08_reg_coef.png">

In [36]:
target_user_movies = set(df_ratings\
    .filter(df_ratings.userid==target_userid)\
    .select('movieid')\
    .rdd.map(lambda row: row.movieid)\
    .collect())

In [37]:
len(target_user_movies)

311

In [38]:
coef = df_ratings.groupby('userid')\
    .agg(F.collect_list(df_ratings.movieid).alias('movieids'))\
    .rdd.map(lambda row: (row.userid, min(1, len(set(row.movieids) & target_user_movies) / 50)))\
    .collect()

In [39]:
coef = sorted(coef)
coef[:3]

[(1, 1), (2, 0.38), (3, 0.16)]

#4. Найдите 30 ближайших пользователей-соседей данного пользователя (`pearson_neighbours`), используя поправленную корреляцию Пирсона: 

<img width="500px" src="http://data.newprolab.com/public-newprolab-com/laba08_pearson_reg_coef.png">

In [40]:
corr_pearson = [coef[i][1] * pearson[i] for i in range(len(pearson))]
corr_pearson[:3]

[0.40686121675947595, 0.12799088872883987, 0.024754587048673921]

In [41]:
top30 =[(k,v) for k,v in sorted(zip(corr_pearson, get_userid_iter()), reverse=True)[:31] if v != target_userid]

In [42]:
top30userids = {v for k,v in top30}

In [43]:
' '.join([str(x) for x in top30userids])

'640 5 268 397 276 916 22 407 663 293 44 429 560 447 453 70 457 334 82 344 345 476 741 749 882 246 632 889 381 638'

#5. Для всех фильмов найдите прогноз оценки по формуле ниже. Здесь *N(a)* — множество пользователей-соседей, *s(a,u)* — мера близости пользователей из предыдущих пунктов, *|s(a,u)|* — модуль меры близости. 

<img width="250px" src="http://data.newprolab.com/public-newprolab-com/laba08_user_user_cf.png">


# ** Ошибиться негде!!! **
```
time: 2017-19-11 22:55
ratings_count / (users_count * films_count) is ok: True
average ratings count per film is ok: True
pearson neighbours intersection for your user: [1.0, True]
length of intersection values is reasonable: True
top10 by pearson intersection: [0.6, False]
file lab08.json found in your home directory: True
average ratings count per user is ok: True
file is readable and has all required fields: True
```

In [44]:
target_user_avg_rating = avg_ratings[target_userid - 1]
target_user_avg_rating
# вообще не влияет на результат

3.1543408360128615

In [45]:
# отбираем только рейтинги для топ 30, по которым полное совпадение с чекером
df_pivot_top = df_ratings.rdd.filter(lambda row: row.userid in top30userids)\
    .toDF().groupBy("movieid").pivot("userid", list(top30userids)).sum("rating")\
    .cache()

In [46]:
# тут ошибиться тоже негде
def predict_rating_row(row):
    row_ = row.asDict()
    movieid = int(row_.pop('movieid'))
    ratings = [(int(k),v) for k,v in row_.items() if v is not None]
    if len(ratings) == 0:
        return target_user_avg_rating
    c = 0
    z = 0
    for userid, rating in ratings:
        uid = userid - 1 # так как нумерация пользователей и фильмов с 1
        sau = corr_pearson[uid]
        rui = rating
        ru = avg_ratings[uid]
        c += sau * (rui - ru)
        z += abs(sau)
    return (target_user_avg_rating + c / z, movieid)

In [47]:
row = df_pivot_top.rdd.take(1)[0]

In [48]:
row

Row(movieid=474, 640=4, 5=None, 268=5, 397=5, 276=5, 916=4, 22=None, 407=3, 663=None, 293=5, 44=4, 429=None, 560=None, 447=3, 453=None, 70=None, 457=5, 334=3, 82=3, 344=None, 345=None, 476=None, 741=None, 749=None, 882=None, 246=None, 632=None, 889=4, 381=None, 638=None)

In [49]:
predict_rating_row(row)

(3.7320291655409452, 474)

In [50]:
predicted_ratings = df_pivot_top.rdd.map(predict_rating_row).collect()

In [51]:
predicted_ratings[:3]

[(3.7320291655409457, 474), (1.9693100618359158, 29), (3.0699764403965322, 26)]

In [238]:
import pandas as pd

In [240]:
df_predicted = pd.DataFrame(predicted_ratings, columns=['prating', 'movieid'])

In [242]:
df_predicted.sort_values(['prating', 'movieid'], ascending=[False, True], inplace=True)

In [244]:
df_predicted.head(n=10)

,prating,movieid
41,4.760621,602
691,4.718758,1589
342,4.702313,114
361,4.634821,115
640,4.634821,906
549,4.565629,653
684,4.548004,199
1010,4.531390,837
962,4.513258,483
98,4.506558,50


In [251]:
pearson_top10 = [int(v) for v in df_predicted['movieid'] if v not in target_user_movies][:10]
sorted(pearson_top10)

[64, 114, 115, 119, 516, 522, 653, 837, 906, 1589]

In [252]:
lab8result = {
    "average_film_ratings": average_film_ratings,
    "average_user_ratings": average_user_ratings,
    "completeness": completeness,
    "pearson_neighbours": sorted(top30userids),
    "pearson_top10": sorted(pearson_top10)
}

In [253]:
with open('../lab08.json', 'w') as fw:
    fw.write(json.dumps(lab8result))

In [254]:
!head ../lab08.json

{"pearson_top10": [64, 114, 115, 119, 516, 522, 653, 837, 906, 1589], "average_user_ratings": 106.04453870625663, "average_film_ratings": 59.45303210463734, "pearson_neighbours": [5, 22, 44, 70, 82, 246, 268, 276, 293, 334, 344, 345, 381, 397, 407, 429, 447, 453, 457, 476, 560, 632, 638, 640, 663, 741, 749, 882, 889, 916], "completeness": 0.06304669364224531}

### **Часть 3. Базовые предикторы:**

#### 1. Глобальное среднее 𝞵 (`average_rating`) по всему датасету. `Сумма всех оценок по всем фильмам / Количество всех оценок по всем фильмам.`

In [450]:
sum_all_ratings = df_ratings.select('rating')\
    .agg(F.sum(df_ratings.rating).alias('sum')).rdd.map(lambda row: row.sum).collect()[0]
sum_all_ratings

352986

In [451]:
num_all_ratings = df_ratings.count()
num_all_ratings

100000

In [452]:
average_rating = sum_all_ratings / num_all_ratings
average_rating

3.52986

#### 2. Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь I<sub>a</sub> — множество фильмов, по которым у пользователя есть рейтинги, а |I<sub>a</sub>| — их количество.

<img width="200px" src="http://data.newprolab.com/public-newprolab-com/laba08s_base_u.png">


In [453]:
#df_pivot_rows = df_ratings.groupBy("userid").pivot("movieid", range(1, num_movies + 1)).sum("rating").cache()

In [454]:
def base_predict_user(row):
    row_ = row.asDict()
    userid = row_.pop('userid')
    values = [v for v in row_.values() if v is not None]
    return (userid, 1 / (len(values) + 10) * sum([rui - average_rating for rui in values]))

In [455]:
bp_users = df_pivot_rows.rdd.map(base_predict_user).collect()

In [456]:
bp_users = sorted(bp_users)
bp_users[:3]

[(1, 0.0775818439716313), (2, 0.1548427777777777), (3, -0.6189443749999995)]

#### 3. Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь U<sub>i</sub> — множество пользователей, которые оценили данный фильм, а |U<sub>i</sub>| — их количество.

<img width="250px" src="http://data.newprolab.com/public-newprolab-com/laba08s_base_i.png">


In [457]:
df_pivot_movies = df_ratings.groupBy("movieid").pivot("userid", range(1, num_users + 1)).sum("rating").cache()

In [458]:
def base_predict_movie(row):
    row_ = row.asDict()
    movieid = row_.pop('movieid')
    ratings = [(int(k), v) for k,v in row_.items() if v is not None]
    return (movieid,\
            1 / (len(ratings) + 25) * sum([rui - bp_users[userid - 1][1] - average_rating for userid, rui in ratings]))

In [459]:
r = df_pivot_movies.take(1)[0]

In [460]:
base_predict_movie(r)

(474, 0.5597371834821095)

In [461]:
bp_movies = df_pivot_movies.rdd.map(base_predict_movie).collect()

In [462]:
bp_movies = sorted(bp_movies)
bp_movies[:3]

[(1, 0.29292488601631034), (2, -0.21303169608613204), (3, -0.3202843059213391)]

#### 4. Базовый предиктор для каждого пользователя и каждого фильма:

<img width="140px" src="http://data.newprolab.com/public-newprolab-com/laba08s_base_ui.png">


In [463]:
bp_users_movies = np.matrix([[average_rating + bu + bi for km, bu in bp_movies] for ku, bi in bp_users])

In [464]:
bp_users_movies.shape

(943, 1682)

In [465]:
bp_users_movies[:5,:5]

matrix([[ 3.90036673,  3.39441015,  3.28715754,  3.58753002,  3.44879883],
        [ 3.97762766,  3.47167108,  3.36441847,  3.66479095,  3.52605977],
        [ 3.20384051,  2.69788393,  2.59063132,  2.8910038 ,  2.75227261],
        [ 4.38994253,  3.88398595,  3.77673334,  4.07710582,  3.93837464],
        [ 3.20264705,  2.69669047,  2.58943786,  2.88981034,  2.75107915]])

### **Часть 4. Item-item CF:**

#### 1. Вычесть из всех рейтингов r<sub>ui</sub> базовый предиктор b<sub>ui</sub> из пункта 4, часть 3 (для всей таблицы рейтингов). Если рейтинга нет, то можно поставить 0.


In [466]:
rui = np.matrix(
    [v for k,v in sorted(df_pivot_rows.rdd.map(lambda row: (row.userid, 
        [v for k,v in sorted([(int(k), v) for k,v in row.asDict().items() if k != 'userid'])])
    ).collect())])

In [467]:
rui.shape

(943, 1682)

In [468]:
blanks = rui == None

In [469]:
blanks

matrix([[False, False, False, ...,  True,  True,  True],
        [False,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [False,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True, False,  True, ...,  True,  True,  True]], dtype=bool)

In [470]:
rui[rui == None] = 0

In [471]:
rui[:5,:5]

matrix([[5, 3, 4, 3, 3],
        [4, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [4, 3, 0, 0, 0]], dtype=object)

In [472]:
rui[1,1] == 0

True

In [473]:
cf = (rui - bp_users_movies).transpose()

In [474]:
cf[rui.transpose() == 0] = 0

In [475]:
cf.shape

(1682, 943)

In [477]:
cf[:5,:5]

matrix([[1.0996332700120584, 0.022372336205911836, 0, 0, 0.7973529518215265],
        [-0.39441014788549955, 0, 0, 0, 0.30330953392396864],
        [0.7128424619497076, 0, 0, 0, 0],
        [-0.5875300200866946, 0, 0, 0, 0],
        [-0.44879883323059255, 0, 0, 0, 0]], dtype=object)

In [478]:
csr = csr_matrix(cf, dtype='float')

In [479]:
def get_csr():
    data = []
    indices = []
    indptr = [0]
    for i in range(rui_.shape[0]):
        ptr = 0
        for j in range(rui_.shape[1]):
            if rui_[i,j] == 0:
                continue
            data.append(cf_[i,j])
            indices.append(j)
            ptr += 1
        indptr.append(ptr)

In [480]:
from scipy.sparse import csr_matrix

In [481]:
#m = csr_matrix((data, indices, indptr), (1682, 943))

#### 2. Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 1, часть 4. Суммирование идет по всем пользователям.

<img width="350px" src="http://data.newprolab.com/public-newprolab-com/laba08s_cosine_items.png">

In [482]:
def custom_cossim(m1, m2):
    c = cf[:,[m1,m2]].prod(axis=1).sum()
    z1 = math.sqrt(cf[:,[m1,m1]].prod(axis=1).sum())
    z2 = math.sqrt(cf[:,[m2,m2]].prod(axis=1).sum())
    return c / (z1 * z2)

In [483]:
#custom_cossim(649,115)

In [484]:
#cossim = np.zeros((1682,1682),dtype='float')

In [485]:
#for i in range(1682):
#    clear_output(wait=True)
#    print(i)
#    for j in range(i, 1682):
#        cossim[i,j] = custom_cossim(i,j)

In [486]:
from sklearn.metrics.pairwise import cosine_similarity

In [487]:
#cossim = cosine_similarity(cf.transpose(),cf.transpose())

In [488]:
cossim = cosine_similarity(csr, csr)

In [489]:
cossim.shape

(1682, 1682)

In [490]:
df_cos = pd.DataFrame(cossim)

In [491]:
df_cos

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,1.000000,-0.003603,-0.012158,-0.035144,0.039609,0.022780,0.021228,0.052695,-0.004018,-0.015940,...,0.077118,0.000000,0.000000,0.000000,0.028896,0.0,0.0,0.0,0.039629,-0.018718
1,-0.003603,1.000000,-0.001984,0.050458,0.002228,-0.003639,-0.016975,0.040146,-0.089526,-0.007177,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.022621,0.016415
2,-0.012158,-0.001984,1.000000,-0.113536,-0.005417,0.060713,-0.024408,-0.094746,-0.016745,-0.039964,...,0.000000,0.000000,0.000000,0.000000,0.183583,0.0,0.0,0.0,0.000000,0.004880
3,-0.035144,0.050458,-0.113536,1.000000,-0.149507,-0.019089,0.005145,0.110511,0.033087,0.010968,...,0.000000,0.000000,-0.115990,-0.115990,0.089827,0.0,0.0,0.0,0.001824,-0.048334
4,0.039609,0.002228,-0.005417,-0.149507,1.000000,-0.031989,-0.015494,0.038203,-0.042447,-0.036512,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.027241
5,0.022780,-0.003639,0.060713,-0.019089,-0.031989,1.000000,-0.049966,-0.076343,0.008348,-0.036051,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
6,0.021228,-0.016975,-0.024408,0.005145,-0.015494,-0.049966,1.000000,0.017618,0.140064,0.022989,...,0.000000,0.018422,0.000000,0.000000,-0.029240,0.0,0.0,0.0,0.041703,-0.021404
7,0.052695,0.040146,-0.094746,0.110511,0.038203,-0.076343,0.017618,1.000000,-0.002634,0.039560,...,0.000000,-0.038699,-0.013646,-0.013646,-0.100956,0.0,0.0,0.0,0.117235,0.000000
8,-0.004018,-0.089526,-0.016745,0.033087,-0.042447,0.008348,0.140064,-0.002634,1.000000,-0.021018,...,0.000000,0.000000,-0.014082,-0.014082,-0.087274,0.0,0.0,0.0,0.041731,-0.078694
9,-0.015940,-0.007177,-0.039964,0.010968,-0.036512,-0.036051,0.022989,0.039560,-0.021018,1.000000,...,0.000000,0.000000,0.078010,0.078010,0.000000,0.0,0.0,0.0,0.000000,0.000000


#### 3. Для каждого фильма, по которому у данного пользователя не стоит рейтинг, найдите:

##### * [a] 30 ближайших фильмов-соседей для этого фильма (среди всех фильмов, а не фильмов, оценённых пользователем).

In [492]:
target_movies = set(range(1, num_movies + 1)) - target_user_movies 

In [493]:
target_movies = list(target_movies)

In [494]:
len(target_user_movies )

311

In [495]:
len(target_movies)

1371

In [496]:
def get_movie_recs(movieid):
    mid = movieid - 1
    recs = [v for k,v in sorted(zip(cossim[mid,:], range(1, num_movies + 1)), reverse=True)[:31] if v != movieid]
    return recs

In [497]:
sorted(zip(cossim[0,:], range(1, num_movies + 1)), reverse=True)[:5]

[(1.0000000000000004, 1),
 (0.18543373535924645, 588),
 (0.14905375454929801, 933),
 (0.14873207780860065, 174),
 (0.14828287200080045, 870)]

In [498]:
len(cossim[1,:])

1682

In [499]:
' '.join([str(x) for x in get_movie_recs(5)])

'1554 581 376 1550 397 1053 857 852 830 911 907 1596 1595 571 839 939 817 912 954 218 842 573 37 838 897 1505 431 441 706 1220'

In [500]:
target_movies_rec = []
for movieid in target_movies:
    target_movies_rec.append(get_movie_recs(movieid))

In [501]:
[' '.join([str(m) for m in recs]) for recs in target_movies_rec[:3]]

['1230 409 1677 282 1231 33 1552 783 1660 1659 1476 1443 1052 1477 67 1408 952 559 1652 860 41 388 421 554 161 1058 628 1518 998 552',
 '1554 581 376 1550 397 1053 857 852 830 911 907 1596 1595 571 839 939 817 912 954 218 842 573 37 838 897 1505 431 441 706 1220',
 '1196 1622 1672 1292 1056 536 1331 1370 1368 1342 1651 1650 1649 1647 1645 1641 1640 1638 1637 1636 1635 1633 1632 1630 113 1623 752 1377 1421 1166']

##### * [b] прогноз оценки пользователя по формуле (базовый предиктор из пункта 4, часть 3).  Здесь *S(i)*- множество фильмов-соседей для фильма *i*, по которым у данного пользователя есть оценка.
<img width="300px" src="http://data.newprolab.com/public-newprolab-com/laba08s_item_item_cf.png">

     Заметим, что суммирование идет только по тем фильмам-соседям, которые оценил пользователь.

In [502]:
def calc_target_rating(movieid, filter_neg=False):
    mid = movieid - 1
    uid = target_userid - 1
    bui = bp_users_movies[uid, mid]
    rec_id = target_movies.index(movieid)
    movies_intersection = set(target_movies_rec[rec_id]) & target_user_movies
    if len(movies_intersection) == 0:
        return (bui, movieid)
    c = 0
    z = 0
    for movieid_temp in movies_intersection:
        tid = movieid_temp - 1
        sij = cossim[mid, tid]
        if filter_neg and sij < 0:
            continue
        ruj = rui[uid, tid]
        buj = bp_users_movies[uid, tid]
        c += sij * (ruj - buj)
        z += abs(sij)
    if c == 0 or z == 0:
        return (bui, movieid)
    return (bui + c / z, movieid)

In [503]:
target_predicts = [calc_target_rating(movieid) for movieid in target_movies]

In [504]:
target_predicts[:3]

[(2.9159658992201551, 3), (2.7135858362361365, 5), (3.2235835742935732, 6)]

#### 4. Рекомендуйте пользователю 10 фильмов (`predicators_top10`) с самыми высокими оценками из пункта 3, часть 4.

In [505]:
top_movies = sorted(target_predicts, reverse=True)[:10]
top_movies

[(5.1387095677591352, 1282),
 (4.7479493107216477, 1127),
 (4.5087942225169853, 126),
 (4.3439919908369591, 83),
 (4.3376583107924365, 1245),
 (4.3223069648664865, 1205),
 (4.3156947459841604, 1113),
 (4.2717504385243874, 1469),
 (4.2602903602084474, 656),
 (4.2546646151244509, 652)]

In [506]:
predicators_top10 = [v for k,v in top_movies]
predicators_top10

[1282, 1127, 126, 83, 1245, 1205, 1113, 1469, 656, 652]

#### 5. При подсчете прогноза по формуле из пункта 3, часть 4 отфильтруйте всех соседей с отрицательной близостью.

In [507]:
target_predicts_pos = [calc_target_rating(movieid, True) for movieid in target_movies]

In [508]:
target_predicts_pos[:3]

[(2.9159658992201551, 3), (2.7135858362361365, 5), (3.2235835742935732, 6)]

#### 6. Рекомендуйте пользователю 10 фильмов (`predicators_positive_top10`) с самыми высокими оценками из пункта 5, часть 4.

In [509]:
top_pos = sorted(target_predicts_pos,reverse=True)[:10]
top_pos

[(5.1387095677591352, 1282),
 (4.7479493107216477, 1127),
 (4.5087942225169853, 126),
 (4.3439919908369591, 83),
 (4.3376583107924365, 1245),
 (4.3223069648664865, 1205),
 (4.3156947459841604, 1113),
 (4.2717504385243874, 1469),
 (4.2602903602084474, 656),
 (4.2546646151244509, 652)]

In [510]:
predicators_positive_top10 = [v for k,v in top_pos]
predicators_positive_top10

[1282, 1127, 126, 83, 1245, 1205, 1113, 1469, 656, 652]

In [511]:
lab8s_result = {
    "average_rating": average_rating,
    "predicators_positive_top10": predicators_positive_top10,
    "predicators_top10": predicators_top10
}

In [512]:
with open('../lab08s.json', 'w') as fw:
    fw.write(json.dumps(lab8s_result))

In [513]:
!head ../lab08s.json

{"predicators_top10": [1282, 1127, 126, 83, 1245, 1205, 1113, 1469, 656, 652], "predicators_positive_top10": [1282, 1127, 126, 83, 1245, 1205, 1113, 1469, 656, 652], "average_rating": 3.52986}

In [ ]:
!head -n3 ml-100k/u.item

movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.

In [ ]:
cols = ["movie id","movie title","release date","video release date","IMDb URL","unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
cols = [('gen_' if i > 5 else '') +\
         re.sub(r'[^\w]', '', col.lower()) for i, col in enumerate(cols)]

In [ ]:
def correct_type(i,d):
    if i == 1 or i == 4: return d
    if i == 2 or i == 3: 
        return datetime.strptime(d, "%d-%b-%Y").date() if d.strip() != '' else None
    return int(d)

In [ ]:
rdd_movies = sc.textFile("/user/valery.baranov/u.item")\
    .map(lambda line: [correct_type(i,d)\
                       for i,d in enumerate(line.split('|'))])\
    .map(lambda ds: Row(**{cols[i]: ds[i] for i in range(len(ds))}))

In [ ]:
df_movies = rdd_movies.toDF(sampleRatio=0.1)

In [ ]:
df_movies = df_movies.withColumn('releaseyear', F.year(df_movies.releasedate))